In [ ]:
import cv2

# 비디오 캡처 객체 생성
cap1 = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(1)

# 이미지 정합에 사용될 변수 초기화
MIN_MATCH_COUNT = 10
detector = cv2.ORB_create()
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
ref_img = None
ref_kp = None
ref_des = None

while True:
    # 2개의 비디오 프레임 읽기
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    
    if not ret1 or not ret2:
        print("비디오 입력 오류")
        break
    
    # 이미지 정합
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # 최초 1회에는 ref_img를 초기화
    if ref_img is None:
        ref_img = gray1
        ref_kp, ref_des = detector.detectAndCompute(ref_img, None)
        continue
    
    # 이미지 정합에 사용할 특징점과 기술자 추출
    kp1, des1 = detector.detectAndCompute(gray1, None)
    kp2, des2 = detector.detectAndCompute(gray2, None)
    
    # BFMatcher를 사용하여 최적의 매칭을 찾기
    matches = matcher.match(ref_des, des1)
    matches = sorted(matches, key=lambda x: x.distance)
    
    # 매칭된 특징점의 개수가 충분하지 않으면 이미지 정합 실패
    if len(matches) < MIN_MATCH_COUNT:
        print("이미지 정합 실패")
        continue
    
    # 매칭된 특징점을 사용하여 Homography 행렬 구하기
    src_pts = np.float32([ref_kp[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp1[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    # 정합된 영상 출력
    result = cv2.warpPerspective(frame2, H, (frame1.shape[1] + frame2.shape[1], frame1.shape[0]))
    result[0:frame1.shape[0], 0:frame1.shape[1]] = frame1
    cv2.imshow("result", result)
    
    # 종료키 확인
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료
cap1.release()
cap2.release()
cv2.destroyAllWindows()